In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import Counter
import pymysql
from konlpy.tag import Okt

# MySQL 연결 설정
conn = pymysql.connect(
    host='127.0.0.1',  
    user='root',       
    password='1234',
    db='yfdb',      
    charset='utf8'     
)
cursor = conn.cursor()

# Okt 객체 생성 형태소 분리 해주는 녀석
okt = Okt()

# 불용어 리스트 생성
stopwords = ['막', '첫', '뭔', '은', '는', '이', '가', '을', '를', '와', '과', 
             '있', '없', '아니', '것', '들', '그', '되', '수', '이', '보', '않', 
             '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', 
             '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', 
             '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', 
             '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', 
             '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', 
             '적', '월', '데', '자신', '안', '어떤', '내', '내', '경우', '명', '생각', 
             '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', 
             '여자', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', 
             '원', '잘', '통하', '소리', '놓']
             
def job(page_num):
    url = f"https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query=%EC%95%A0%ED%94%8C={page_num}"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    news_titles = soup.select('.news_tit')

    titles = [title.get_text() for title in news_titles]
    return titles

news_titles = []
for i in range(1, 101, 10):
    news_titles.extend(job(i))

df = pd.DataFrame(news_titles, columns=['title'])

words = []
for title in df['title']:
    # 형태소 분리
    temp = okt.morphs(title)
    # 불용어 제거
    temp = [word for word in temp if word not in stopwords and len(word) > 1]
    words.extend(temp)

counter = Counter(words)

word_counts = pd.DataFrame.from_dict(counter, orient='index').reset_index()

# my sql 저장
for row in word_counts.itertuples():
    word = row[1]
    count = row[2]
    sql = "INSERT INTO word_counts (word, count) VALUES (%s, %s)"
    cursor.execute(sql, (word, count))

# MySQL 변경사항 반영
conn.commit()

# MySQL 연결 close
conn.close()